# County-Year Metrics Aggregation (TX + CO)



In [2]:
# County-Year Metrics Aggregation (TX + CO)

import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

db_url = os.getenv("OG_IMPACT_DB_URL")
engine = create_engine(db_url)

# Helper: aggregate by county + year using FIPS or name
def summarize_by_county_year(table, schema, date_col, id_col, county_col="county_fips", extra_where=None, fips_join=False, state_abbr=None):
    conditions = [f"{date_col} IS NOT NULL", f"{county_col} IS NOT NULL"]
    if extra_where:
        conditions.append(extra_where)

    if fips_join and state_abbr:
        query = f"""
            SELECT
                c.county_fips AS fips,
                EXTRACT(YEAR FROM {table}.{date_col}::timestamp)::INT AS year,
                COUNT({table}.{id_col}) AS count
            FROM {schema}.{table}
            JOIN public.counties c
              ON LOWER({table}.{county_col}) = LOWER(c.name)
             AND c.state = '{state_abbr}'
            WHERE {' AND '.join(conditions)}
            GROUP BY c.county_fips, year
            ORDER BY c.county_fips, year
        """
    else:
        query = f"""
            SELECT
                {county_col} AS fips,
                EXTRACT(YEAR FROM {date_col}::timestamp)::INT AS year,
                COUNT({id_col}) AS count
            FROM {schema}.{table}
            WHERE {' AND '.join(conditions)}
            GROUP BY {county_col}, year
            ORDER BY {county_col}, year
        """

    return pd.read_sql(query, engine)


# Texas: violations and inspections (use county name)
tx_violations = summarize_by_county_year("violations", "tx", "violation_disc_date", "id", county_col="county")
tx_inspections = summarize_by_county_year("inspections", "tx", "inspection_date", "id", county_col="county")

# Colorado: wells — filter out invalid dates
co_wells = summarize_by_county_year(
    "wells_joined", "co", '"Spud_Date"', '"API"',
    county_col="name",
    extra_where='"Spud_Date" !~ \'^0000|^9999|00/00/0000\''
)
co_violations = summarize_by_county_year(
    "violations", "co", '"DateOfViolation"', '"DocumentNumber"',
    county_col='"County"', fips_join=True, state_abbr="CO"
)

co_inspections = summarize_by_county_year(
    "inspections", "co", '"InspectionDate"', '"DocNum"',
    county_col='"County"', fips_join=True, state_abbr="CO"
)

# Concatenate all and tag by source/type
all_counts = pd.concat([
    tx_violations.assign(source="TX", type="violation"),
    tx_inspections.assign(source="TX", type="inspection"),
    co_wells.assign(source="CO", type="well"),
    co_violations.assign(source="CO", type="violation"),
    co_inspections.assign(source="CO", type="inspection")
])


# Write to public schema
all_counts.to_sql("county_year_counts", engine, schema="public", if_exists="replace", index=False)
print("✅ County-year metrics table written to public.county_year_counts")


✅ County-year metrics table written to public.county_year_counts
